---
title: 統計的学習
format: html
lang: ja
execute: 
  eval: true
ascii: false
keep-ipynb: true
jupyter:
  jupytext:
    formats: 'ipynb,qmd'
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.14.5
  kernelspec:
    display_name: R
    language: R
    name: ir
---

In [ ]:
#| eval: false
#| echo: true
install.packages("tidymodels")

## 教師あり学習: ペンギンデータの性別を分類するモデル

In [ ]:
#| label: ペンギンデータの読み込み
penguins <- 
  readr::read_csv("https://raw.githubusercontent.com/allisonhorst/palmerpenguins/main/inst/extdata/penguins.csv",
                  col_types = "ccddddcd")

In [ ]:
library(tidymodels)
library(dplyr, warn.conflicts = FALSE)
penguins_df <- 
  penguins |> 
  # 性別が欠損値となる行を削除
  filter(!is.na(sex)) |> 
  # モデル中で利用しない変数の除外
  select(!c(year, island))

### データの可視化

In [ ]:
library(ggplot2)

penguins_df |> 
  ggplot(aes(flipper_length_mm, bill_length_mm, 
             color = body_mass_g, 
             shape = sex,
             size = body_mass_g)) +
  geom_point(alpha = 0.5) +
  scale_color_viridis_c() +
  facet_wrap(~species)

### データ分割


データを訓練データとテストデータに分割する。各データに偏りがないように性別による層化抽出を行う。

In [ ]:
set.seed(123)
# 性別を層とする層化抽出
# ref) 第7回「データと確率」
penguin_split <- 
  # 訓練データとテストデータの件数の比率をprop引数で指定
  initial_split(penguins_df, strata = sex, prop = 3/4)
penguin_train <- training(penguin_split)
penguin_test <- testing(penguin_split)

元のデータ、訓練データ、テストデータの性別の頻度を確認する。

In [ ]:
count(penguins_df, sex)
count(penguin_train, sex)
count(penguin_test, sex)

### 特徴量エンジニアリング

次の特徴量エンジニアリングを行う。

- データの標準化
- カテゴリ変数のダミー変数化

In [ ]:
penguins_rec <-
  recipe(formula = sex ~ ., 
         data = penguin_train) |> 
  step_normalize(all_numeric()) |> 
  step_dummy(all_nominal(), -sex, one_hot = TRUE)

In [ ]:
# 訓練データがどのように変換されるかを確認する
head(bake(prep(penguins_rec), new_data = NULL))

### ロジスティック回帰

リンク関数がロジットで、誤差構造が二項分布の場合の一般化線形モデルを構築する。

In [ ]:
glm_spec <- 
  logistic_reg() |>
  set_engine("glm")

In [ ]:
penguin_wf <- 
  workflow() |> 
  add_model(glm_spec) |> 
  add_recipe(penguins_rec)

訓練データを用いてモデルを学習する。

In [ ]:
penguins_fit <-
  penguin_wf |> 
  fit(data = penguin_train)

penguins_fit

In [ ]:
#| eval: false
#| echo: true
# 同じ結果となるのを確認
glm(sex ~ .,
    data = bake(prep(penguins_rec), new_data = NULL), 
    family = binomial(link = "logit"))

学習したモデルを用いてテストデータから性別を予測する。

In [ ]:
head(predict(penguins_fit, new_data = penguin_test))

In [ ]:
# 分類を誤ったデータを確認
predict(penguins_fit, new_data = penguin_test) |> 
  bind_cols(penguin_test) |> 
  filter(.pred_class != sex)

### 性能評価

テストデータを用いて学習したモデルの性能を評価する。

In [ ]:
# 評価関数accuracy()により正解率を求める
# 正解率はモデルが正しく予測したデータの割合を示す
augment(penguins_fit, new_data = penguin_test |> 
          mutate(sex = as.factor(sex))) |> 
  accuracy(truth = sex, estimate = .pred_class)

## 教師なし学習: k-平均法

In [ ]:
df_animal_tiny <-
  read.csv("https://raw.githubusercontent.com/uribo/tokupon_ds/main/data-raw/tokushima_zoo_animals22.csv") |> 
  select(body_length_cm, weight_kg) |> 
  # 欠損値を含む行を削除
  tidyr::drop_na()

In [ ]:
# データの特徴からグループ分けを行いたい
df_animal_tiny |> 
  ggplot() +
  aes(body_length_cm, weight_kg) +
  geom_point()

tidymodelsの枠組みでクラスタリングを扱うtidyclustパッケージで行う。


In [ ]:
#| eval: false
#| echo: true
install.packages(c("tidyclust"))

In [ ]:
library(tidyclust)
kmeans_spec <-
  k_means(num_clusters = 4) |>
  set_engine("stats")
kmeans_spec_fit <-
  kmeans_spec |>
  fit(~ ., data = df_animal_tiny)
kmeans_spec_fit

クラスタリングの結果をプロットして確認する。

In [ ]:
augment(kmeans_spec_fit,
        df_animal_tiny) |>
  ggplot() +
  aes(x = body_length_cm, y = weight_kg, colour = .pred_cluster) +
  geom_point()